# Loading training data

In [1]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        X.append(row)

y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        y.append(row.replace("\n",""))


In [2]:
X = np.array(X)
y = np.array(y)

# Data vectorisation

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

# Data test loading and vectorisation

In [4]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))

test = np.asarray(test)
test_dtm = vect.transform(test)

In [5]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

# Model implementation and training
## Measurements calculation and results

In [6]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, f1_score, jaccard_score, classification_report

models = [
    SGDClassifier(loss='log', penalty='l2', alpha=0.001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                    tol=0.001, epsilon=0.1, n_jobs=-1, validation_fraction=0.1, 
                    n_iter_no_change=5)

]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    predict_label = model.predict(test_dtm)
    acc = accuracy_score(predict_label,test_label)
    f1 = f1_score(predict_label, test_label, average = 'weighted')
    cm = multilabel_confusion_matrix(test_label,predict_label)
    jaccard = jaccard_score(test_label, predict_label, average='micro')
    print(classification_report(test_label, predict_label))
    print ("la matrice de confusion : ")
    print(cm)
    entries.append((model_name, acc, f1, jaccard))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy', 'f1', 'jaccard'])

/home/julien/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/julien/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.26      0.88      0.40     10798
           1       0.28      0.07      0.12      4830
          10       0.15      0.09      0.11      1432
          11       0.70      0.16      0.26      1949
          12       0.40      0.18      0.25      1265
          13       0.00      0.00      0.00      1114
          14       0.00      0.00      0.00      1306
          15       0.33      0.00      0.00      1244
          16       0.00      0.00      0.00      1153
          17       0.68      0.54      0.60      1545
          18       0.50      0.00      0.00      2417
          19       0.00      0.00      0.00      1010
           2       0.28      0.42      0.33      4534
           3       0.00      0.00      0.00      2605
           4       0.57      0.27      0.37      3716
           5       0.11      0.00      0.00      1613
           6       0.50      0.01      0.03      1996
           7       0.46    

In [7]:
print(cv_df)

      model_name  accuracy        f1   jaccard
0  SGDClassifier     0.287  0.377536  0.167542


# Grid Search use to find the best parameters for this model

In [19]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

# Best with 'alpha': 0.001, 'loss': 'log', 'penalty': 'l2'
parameters = {'loss':('hinge', 'log', 'modified_huber'), 'penalty':('l2', 'elasticnet'), 'alpha':[0.0001, 0.001], }
sgd = SGDClassifier(l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                    tol=0.001, epsilon=0.1, n_jobs=-1, validation_fraction=0.1, 
                    n_iter_no_change=5)

clf = GridSearchCV(sgd, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)

print(clf.cv_results_)

{'mean_fit_time': array([0.10695187, 0.10583456, 0.10537783, 0.10672967, 0.10555379,
       0.10584307, 0.10597563, 0.10581088, 0.10659146, 0.1049238 ,
       0.10333085, 0.10419432]), 'std_fit_time': array([0.0003163 , 0.00145405, 0.0019632 , 0.00246297, 0.00088142,
       0.00066537, 0.0011465 , 0.00253886, 0.002998  , 0.00133233,
       0.00012408, 0.00048818]), 'mean_score_time': array([0.00113877, 0.00090917, 0.00089359, 0.00091108, 0.00115363,
       0.00109688, 0.00089653, 0.00084003, 0.00097855, 0.00072837,
       0.00098077, 0.00131488]), 'std_score_time': array([2.36474803e-04, 2.43973421e-04, 2.35064518e-04, 6.40423054e-05,
       3.76382188e-04, 1.83180408e-04, 2.25658396e-04, 2.32812972e-04,
       2.11573202e-04, 1.22644674e-04, 2.65906545e-04, 1.95100804e-04]), 'param_alpha': masked_array(data=[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.001,
                   0.001, 0.001, 0.001, 0.001, 0.001],
             mask=[False, False, False, False, False, False, False, F

/home/julien/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
